<a href="https://colab.research.google.com/github/CanteroAlann/machine-learning-projects/blob/main/TrabajoPractico2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import string
import math as mt 

In [ ]:
# create the Spark Session
spark= SparkSession \
    .builder \
    .getOrCreate()
# create the Spark Context
sc= spark.sparkContext

In [ ]:
df_review = pd.read_csv('/content/drive/MyDrive/tp1_orga/review-002.csv')
df_review.sample(frac=0.4).to_parquet('review.parquet')
df_user = pd.read_csv('/content/drive/MyDrive/tp1_orga/user-001.csv')
df_user.sample(frac=0.3).to_parquet('user.parquet')

S17 (). La antigüedad promedio de los usuarios y el nombre del usuario más antiguo cuyas última review contenga la palabra ‘pizza’

In [ ]:
# cargo dataset de usuarios
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('user.parquet', header=True, inferSchema=True, multiLine=True)
user_rdd = df.rdd
# cargo dataset de reviews
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('review.parquet', header=True, inferSchema=True, multiLine=True)
review_rdd = df.rdd

In [ ]:
import datetime
from datetime import date
def antiguedad(fecha):
  return (pd.to_datetime(date.today()) - pd.to_datetime(fecha, errors='coerce', format='%Y-%m-%d %H:%M:%S')).days

In [ ]:
# mapeo de la siguiente manera 
# x[0] = user_id ; x[1] = name ; x[3] = antiquity 
user_with_antiquity=user_rdd.map(lambda x : (x[0],(x[1] ,antiguedad(x[3])))).cache()

In [ ]:
# funcion para calcular la antiguedad en años, meses y dias 
def func_date(number_of_days):
  # Calculating years
  years = number_of_days // 365
  # Calculating months
  months = (number_of_days - years *365) // 30
  # Calculating days
  days = mt.floor((number_of_days - years * 365 - months*30))
  # Displaying results
  print("Years = ", years)
  print("Months = ", months)
  print("Days = ", days)

In [ ]:
# calculo la antiguedad promedio 
func_date(user_with_antiquity.map(lambda x : (x[1][1])).mean())

In [ ]:
def cont_pizza(string):
  return 'pizza' in str(string)

In [ ]:
# mapeo de la siguiente manera 
# x[0] = user_id
# x[7] = texto de la review 
# x[8] = fecha de la review 
# luego uso reduceByKey para quedarme con la review mas reciente por usuario y luego filtro aquellas reviews que contienen 'pizza'
last_review_with_pizza = review_rdd.map(lambda x :(x[0],(x[7],x[8]))).reduceByKey(lambda x,y : ((x[0] if x[1] > y[1] else y[0]),(x[1] if x[1]>y[1] else y[1]))).filter(lambda x : cont_pizza(x[1][0]))

In [ ]:
# ahora hago un inner join para quedarme con los usuarios cuyas ultimas reviews contienen "pizza"
# luego hago un reduce para quedarme con el usuario mas antiguo
user = user_with_antiquity.join(last_review_with_pizza).reduce(lambda x,y : x if x[1][0][1] > y[1][0][1] else y)
# Rta final
print("el usuario mas antiguo cuya review contiene 'pizza' es :",user[1][0][0])

el usuario mas antiguo cuya review contiene 'pizza' es : Gioia


S20 (). Buscar la distancia mínima que existe entre dos locales de McDonald ‘s en el estado de LA (que no comparten dirección). ¿Cuál es la distancia?¿A qué ciudad/es pertenecen? ¿Cuáles son las direcciones de los locales?


In [ ]:
df_business = pd.read_csv('/content/drive/MyDrive/tp1_orga/business.csv')
df_business.to_parquet('business.parquet')

In [ ]:
# cargo dataset de negocios 
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('business.parquet', header=True, inferSchema=True, multiLine=True)
business_rdd = df.rdd

In [ ]:
# funcion para calcular la distancia 
import math 
def distancia12(lat1, lon1, lat2, lon2):

    lat1rad = math.radians(lat1)
    lon1rad = math.radians(lon1)

    lat2rad = math.radians(lat2)
    lon2rad = math.radians(lon2)

    #  Si las latitudes y longitudes son iguales se encuentran en el mismo sitio geografico
    if lat1rad == lat2rad and lon1rad == lon2rad:
        return 0

    #  Calculo de la distancia P1 a P2
    a = math.sin(lat1rad)*math.sin(lat2rad)
    b = math.cos(lat1rad)*math.cos(lat2rad)*math.cos(lon2rad - lon1rad)
    D = math.acos(a + b)  

    d = 111.18*math.degrees(D) 
    
    return d

In [ ]:
# filtro por las condiciones pedidas por el enunciado y luego mapeo de la siguiente manera :
# x[0] = business_id ; x[2] = address ; x[3] = city ; x[6] = latitude ; x[7] = longitude
aux_rdd = business_rdd.filter(lambda x : x[4] == 'LA').filter(lambda x : x[1]=="McDonald's").map(lambda x : (x[0],x[2],x[3],x[6],x[7])).cache()

In [ ]:
# producto cartesiano 
mc_rdd = aux_rdd.cartesian(aux_rdd)
# rdd auxiliar donde hago un mapeo para obtener lo siguiente :
# x[0][1] = address 1 ; x[0][2] = city 1 ; x[1][1] = address 2 ; x[1][2] = city 2 
cc = mc_rdd.map(lambda x : (x[0][1],x[0][2],x[1][1],x[1][2],distancia12(x[0][3],x[0][4],x[1][3],x[1][4])))

In [ ]:
# filtro aquellas distancias que son 0 y luego busco los mcdonald's mas cercanos 
mcdonals_mas_cercanos = cc.filter(lambda x : x[4] > 0).reduce(lambda x,y : x if x[4] < y[4] else y)
# respuesta final 
print("Direccion 1 : ",mcdonals_mas_cercanos[0]," ; Direccion 2 : ",mcdonals_mas_cercanos[2]," ; Ciudad : ",mcdonals_mas_cercanos[1]," ; Distancia:",mcdonals_mas_cercanos[4],"km")

Direccion 1 :  1212 Stumpf Blvd  ; Direccion 2 :  501 Westbank Expy  ; Ciudad :  Gretna  ; Distancia: 1.1583698947020564 km


S32 (). Yelp tiene la teoría de que quienes hacen las reviews más útiles en la plataforma son aquellos usuarios con más antigüedad. Para probar está hipótesis para cada review consiga la suma total de sus votos (funny + cool + useful) y correlacionela con la antigüedad del usuario al momento de hacer la review.

In [ ]:
df_review = pd.read_csv('/content/drive/MyDrive/tp1_orga/review-002.csv')
df_review.sample(frac=0.2).to_parquet('review.parquet')

In [ ]:
df_user = pd.read_csv('/content/drive/MyDrive/tp1_orga/user-001.csv')
df_user.sample(frac=0.2).to_parquet('user.parquet')

In [ ]:
# cargo dataset de usuarios
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('user.parquet', header=True, inferSchema=True, multiLine=True)
user_rdd = df.rdd
# cargo dataset de reviews
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('review.parquet', header=True, inferSchema=True, multiLine=True)
review_rdd = df.rdd

In [ ]:
# funcion para obtener la suma de votos 
def sum_(x,y,z):
  return float(x) + float(y) + float(z)

In [ ]:
# mapeo de la siguiente manera 
# x[1] = user_id ; x[4],x[5],x[6] = usefull,funny,cool ; x[8] = date
list_user_reviews =review_rdd.map(lambda x : (x[1],(sum_(x[4],x[5],x[6]),x[8])))
# mapeo de la siguiente manera 
# x[0]= user_id ; x[3] = yelping_since
list_users = user_rdd.map(lambda x : (x[0],x[3]))

In [ ]:
# inner join para asi unir cada user_id con su respectiva fecha de ingreso 
usefull_rdd = list_users.join(list_user_reviews)

In [ ]:
# funcion para obtener la antiguedad del usuario al momento de hacer la review 
def antiguedad_usuario(fecha_del_usuario,fecha_de_la_review):
  return (pd.to_datetime(fecha_de_la_review, errors='coerce', format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(fecha_del_usuario, errors='coerce', format='%Y-%m-%d %H:%M:%S')).days

In [ ]:
# ahora mapeo para quedarme con la antiguedad del usuario al momento de hacer la review 
# x[0] = user_id ; x[1][1][0] = suma de votos ; x[1][0] = fecha de ingreso del usuario ; x[1][1][1] = fecha de la review 
prom_rdd = usefull_rdd.map(lambda x: (x[0],x[1][1][0],antiguedad_usuario(x[1][0],x[1][1][1]))).cache()

In [ ]:
# calculo el promedio de la suma de votos 
prom_votos = prom_rdd.map(lambda x: (x[1])).mean()
# calculo el promedio de la antiguedad de los usuarios
prom_antiguedad = prom_rdd.map(lambda x : (x[2])).mean() 

Ahora mapeo para obtener los siguientes terminos de la siguiente formula 

\begin{align}
        {\displaystyle r_{xy}={\frac {\sum _{i=1}^{n}\left(x_{i}-{\bar {x}}\right)\left(y_{i}-{\bar {y}}\right)}{{\sqrt {\sum _{i=1}^{n}\left(x_{i}-{\bar {x}}\right)^{2}}}{\sqrt {\sum _{i=1}^{n}\left(y_{i}-{\bar {y}}\right)^{2}}}}}}
    \end{align}
 donde

 (x[1]-prom_votos)*(x[2]-prom_antiguedad)  = $(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})$

 (x[1]-prom_votos)*(x[1]-prom_votos) = $(x_{i}-{\bar {x}})^{2}$

 (x[2]-prom_antiguedad)*(x[2]-prom_antiguedad) = $(y_{i}-{\bar {y}})^{2}$

In [ ]:
# con el reduce realizo cada sumatoria de la formula anterior 
x = prom_rdd.map(lambda x: ((x[1]-prom_votos)*(x[2]-prom_antiguedad),(x[1]-prom_votos)*(x[1]-prom_votos),(x[2]-prom_antiguedad)*(x[2]-prom_antiguedad))).reduce(lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2]))

In [ ]:
# por ultimo hago las raices y la division para obtener el resultado final 
import math as mt 
corr = (x[0]/(mt.sqrt(x[1])*mt.sqrt(x[2])))
corr